# Using Bike Collision and Other Data to Predict Car Accidents #
## Data Collection and Cleaning ##

Harvard University has collected a number of Boston Area data sets ffor their Boston Area Research Intitiative. The key data for this project about predicting bike accidents was derived from Boston Police Department incident narrative reports, as organized and compiled by Dahianna Lobez and partners ad BPD, BARI, and the Boston Cyclists Union.

This data is for collisions that occured between 2009 and 2012.

### Bike Collision Data ###

In [1]:
import pandas as pd
import geopandas as gpd
import math
from shapely.geometry import Point, LineString, Polygon
from pyproj import Proj, transform
from matplotlib import pyplot as plt

In [2]:
bike_df_orig = pd.read_excel('data/Final_Bike_Collision_Database.xlsx')
bike_df_orig.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1808 entries, 0 to 1807
Data columns (total 54 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   ID          1808 non-null   object        
 1   YEAR        1797 non-null   float64       
 2   DATE        1797 non-null   datetime64[ns]
 3   DAY_WEEK    1795 non-null   object        
 4   TIME        1797 non-null   object        
 5   TYPE        1795 non-null   object        
 6   SOURCE      1699 non-null   object        
 7   XFINAL      1731 non-null   float64       
 8   Xkm         1730 non-null   float64       
 9   YFINAL      1731 non-null   float64       
 10  Ykm         1731 non-null   float64       
 11  Address     1730 non-null   object        
 12  Main        1731 non-null   float64       
 13  RoadType    1726 non-null   object        
 14  ISINTERSEC  1807 non-null   float64       
 15  TRACT       1731 non-null   float64       
 16  CouncilDIS  1731 non-nul

**Xkm** and **Ykm** encode the location of the accident. For this project I'm not interested in accidents that don't have a location, so I'll drop the rows with missing locations.

I also want to make sure there aren't outliers in the location data. The Xkm and Ykm values are meter coordinates in a Massachusetts-specific geographic system. I'll convert them to latitude and logitude and make sure that they don't exceed the map of Boston.

In [3]:
bike_df = pd.DataFrame(bike_df_orig[(bike_df_orig.Xkm > 0) & (bike_df_orig.Ykm > 0)])

# convert the bike data locations into latitude and longitudes and find the geographic extent
inProj = Proj('epsg:26986')
outProj = Proj('epsg:4326')
xmax,ymax = bike_df['Xkm'].max(),bike_df['Ykm'].max()
xmax2,ymax2 = transform(inProj,outProj,xmax,ymax)
print ('Upper Right: ')
print (xmax2,ymax2)

xmin,ymin = bike_df['Xkm'].min(),bike_df['Ykm'].min()
xmin2,ymin2 = transform(inProj,outProj,xmin,ymin)

print('Bottom Right:')
print(xmin2, ymax2)

print('Top Right: ')
print(xmax2, ymin2)

print ('Bottom Left: ')
print (xmin2,ymin2)

Upper Right: 
42.385920811282396 -71.00003640589748
Bottom Right:
42.23530626734878 -71.00003640589748
Top Right: 
42.385920811282396 -71.17443656586595
Bottom Left: 
42.23530626734878 -71.17443656586595


The other columns we may be interested in are RoadType and ISINTERSECT but we can find that data from our other data sets as well, so I'll consider data cleaning on Bike Collision data done at this point.

In [4]:
#output saved bike info for later use
bike_df.to_csv(r'data/cleaned_data/bike_accident_clean.csv')

### Car Accident Data ###
Car accident crash data is from kaggle. https://www.kaggle.com/sobhanmoosavi/us-accidents and shows car accidents in 2019.

In [5]:
car_df = pd.read_csv('data/US_Accidents_Dec19.csv')
len(car_df)

2974335

This is a massive file with nearly 3 million rows and that covers the whole of the US, so the first task is to limit it to the same area as the bike accidents above.

In [6]:
# To capture accidents that may have happened at the edges of the bike collision area
# keep accidents that either start or end within the area
car_df = car_df[((car_df['Start_Lat'] <= xmax2) & (car_df['Start_Lat'] >= xmin2) & 
                 (car_df['Start_Lng'] <= ymax2) & (car_df['Start_Lng'] >= ymin2) |
                (car_df['End_Lat'] <= xmax2) & (car_df['End_Lat'] >= xmin2) & 
                 (car_df['End_Lng'] <= ymax2) & (car_df['End_Lng'] >= ymin2))]

# Re-index because we've removed a lot of values
car_df = car_df.reset_index()

len(car_df)

6343

No other cleaning is necessary on this data, since the only thing I'm interested in is accident locations.

The Mass DOT geospatial system is convenient for this project, so I'll add new columns to contain that data.

In [ ]:
inProj = Proj('epsg:4326')
outProj = Proj('epsg:26986')

#loop through car datafram to create a list of lists containing the new coordinates
points_list = []
for index, row in car_df.iterrows():
    points_row = []
    start_points = list(transform(inProj,outProj,row['Start_Lat'],row['Start_Lng']))
    points_row.append(start_points[0])
    points_row.append(start_points[1])
    end_points = list(transform(inProj,outProj,row['End_Lat'],row['End_Lng']))
    points_row.append(end_points[0])
    points_row.append(end_points[1])
    points_list.append(points_row)

converted_car_df = pd.DataFrame(points_list, columns=['Start_Xkm', 'Start_Ykm', 'End_Xkm', 'End_Ykm'])
len(converted_car_df)
#car_df[['Start_Xkm', 'Start_Ykm']] = transform(inProj,outProj,car_df['Start_Lat'],car_df['Start_Lng'])



That was an extremely time-consuming conversion. Here I've outputted the converted data for posterity so I don't have to do that again. Then I'm adding these as columns to the car data, and outputting that as well.

In [ ]:
#output convered car accident locations for later use
converted_car_df.to_csv(r'data/cleaned_data/converted_car_locs.csv')



In [ ]:
car_df = pd.concat([car_df, converted_car_df], axis=1)
car_df.head()

In [ ]:
car_df.to_csv(r'data/cleaned_data/car_accidents_cleaned.csv')

### Road Data ###
Mass DOT has a database of road data in a .gpd file (https://geo-massdot.opendata.arcgis.com/datasets/road-inventory-2018). This has geometric data about the roads which will be very helpful, and a lot of qualitative road data with a code book [here](RI-DataDictionary.pdf).

I'm going to read in the full roads database, trim to only Boston data, and see if I can find the missing values.

In [ ]:
roads_2018 = gpd.read_file('data\RoadInv2018.gdb', layer=0)
roads_2018.info()

To limit roads_2018 to only boston roads, I'll use geopandas functions to remove roads that aren't in the bike accident extent.

In [ ]:
# Create shape that contains all bike accidents using mins and maxes from above
bike_shape = Polygon([(xmin, ymin), (xmin, ymax), (xmax, ymax), (xmax, ymin)])

In [ ]:
# Rename geometry column so I don't lose it during the overlay
roads_2018 = roads_2018.rename(columns={'geometry': 'road_shape'})

In [ ]:
# In order to perform an interection, I need a column in roads_2018 that contains polygons
roads_2018 = roads_2018.set_geometry('road_shape')
roads_2018['buffer'] = roads_2018['road_shape'].buffer(50)
roads_2018 = roads_2018.set_geometry('buffer')

In [ ]:
# Create GeoDataFrame out of bike shape to enable overlay
bike_df = gpd.GeoDataFrame({ 'shape': [bike_shape]})
bike_df = bike_df.set_geometry('shape')

In [ ]:
rd_gdf = gpd.overlay(bike_df, roads_2018, how='intersection')

In [ ]:
len(rd_gdf)

In [ ]:
rd_gdf.info()

Spot checking some values for outliers.


In [ ]:
rd_gdf['Surface_Wd'].hist()

In [ ]:
rd_gdf['Med_Width'].hist()

250m width for the road median seems like it might be an outlier, but it's not impossible on a highway. I may want to remove highways, but we'll see, and they don't make up many of the roads in Boston anyway.

In [ ]:
# output to file
rd_gdf.to_csv(r'data/cleaned_data/boston_roads_gpd.csv')